In [1]:
# -----------------------------
# 0️⃣ Install required packages
# -----------------------------
# !pip install langchain-groq langgraph python-dotenv

# -----------------------------
# 1️⃣ Load .env and Groq API Key
# -----------------------------
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("GROQ_API_KEY not found in .env file!")

# -----------------------------
# 2️⃣ Imports & LLM Setup
# -----------------------------
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq

# Initialize Groq LLM (no client_options)
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

# -----------------------------
# 3️⃣ Sub-Agent: Summarization
# -----------------------------
class SummarizationState(TypedDict):
    input_text: str
    output_text: str

def summarization_node(state: SummarizationState) -> SummarizationState:
    text = state["input_text"]
    response = llm.invoke(f"Summarize clearly:\n\n{text}")
    state["output_text"] = response.content
    return state

# Build Summarization Agent Graph
summarization_agent = StateGraph(SummarizationState)
summarization_agent.add_node("summarize", summarization_node)
summarization_agent.set_entry_point("summarize")
summarization_agent.add_edge("summarize", END)
summarization_agent = summarization_agent.compile()

# Sub-agent registry
SUB_AGENTS = {"summarize": summarization_agent}

# Task delegation tool
def delegate_task(agent_name: str, input_text: str) -> str:
    if agent_name not in SUB_AGENTS:
        raise ValueError(f"Unknown sub-agent: {agent_name}")

    initial_state = {"input_text": input_text, "output_text": ""}
    final_state = SUB_AGENTS[agent_name].invoke(initial_state)
    return final_state["output_text"]

# -----------------------------
# 4️⃣ Main Agent Workflow
# -----------------------------
class MainAgentState(TypedDict):
    messages: List[HumanMessage]
    todos: List[Dict[str, Any]]
    files: Dict[str, str]

# Planner Node: create dynamic todos
def create_todos(state: MainAgentState) -> MainAgentState:
    text = state["messages"][-1].content
    chunks = [s.strip() for s in text.split('.') if s.strip()]
    todos = []
    for i, chunk in enumerate(chunks):
        todos.append({
            "id": i + 1,
            "task_type": "summarize",  # Could expand to other sub-agent tasks
            "done": False,
            "input_text": chunk
        })
    state["todos"] = todos
    return state

# Supervisor Node: delegate todos
def supervisor_node(state: MainAgentState) -> MainAgentState:
    combined_result = ""
    for todo in state["todos"]:
        if not todo["done"]:
            result = delegate_task(todo["task_type"], todo["input_text"])

            filename = f"{todo['task_type']}_{todo['id']}.txt"
            state["files"][filename] = result

            combined_result += result + " "
            todo["done"] = True

    state["files"]["combined_output.txt"] = combined_result.strip()
    return state

# -----------------------------
# 5️⃣ Build Main Agent Graph
# -----------------------------
main_graph = StateGraph(MainAgentState)
main_graph.add_node("planner", create_todos)
main_graph.add_node("supervisor", supervisor_node)
main_graph.set_entry_point("planner")
main_graph.add_edge("planner", "supervisor")
main_graph.add_edge("supervisor", END)
main_graph = main_graph.compile()

# -----------------------------
# 6️⃣ Run the Agent
# -----------------------------
initial_state = {
    "messages": [
        HumanMessage(
            content=(
                "Artificial Intelligence enables machines to learn from data, "
                "make decisions, and continuously improve without explicit programming. "
                "It is applied in healthcare, finance, robotics, and many other fields."
            )
        )
    ],
    "todos": [],
    "files": {}
}

final_state = main_graph.invoke(initial_state)

# -----------------------------
# 7️⃣ Display Results
# -----------------------------
print("✅ TODO LIST STATUS\n")
for todo in final_state["todos"]:
    status = "✅ Done" if todo["done"] else "❌ Pending"
    print(f"Todo {todo['id']} - {todo['task_type']}: {status}")

print("\n✅ FILES OUTPUT\n")
for file, content in final_state["files"].items():
    print(f"{file}:\n{content}\n")


✅ TODO LIST STATUS

Todo 1 - summarize: ✅ Done
Todo 2 - summarize: ✅ Done

✅ FILES OUTPUT

summarize_1.txt:
Here's a clear summary:

**Artificial Intelligence (AI)**

* Enables machines to learn from data
* Allows machines to make decisions
* Enables continuous improvement without explicit programming

In simpler terms, AI helps machines become smarter and more efficient by learning from data and adapting to new situations.

summarize_2.txt:
Machine learning is applied in various fields, including:

1. Healthcare
2. Finance
3. Robotics
4. And many others.

combined_output.txt:
Here's a clear summary:

**Artificial Intelligence (AI)**

* Enables machines to learn from data
* Allows machines to make decisions
* Enables continuous improvement without explicit programming

In simpler terms, AI helps machines become smarter and more efficient by learning from data and adapting to new situations. Machine learning is applied in various fields, including:

1. Healthcare
2. Finance
3. Robotics
